In this script we investigate a cleaner way of eyeballing vfields to remove confusion of having a long list of files.

In [45]:
import os 
import numpy as np
import pandas as pd
import subprocess
import time
import signal

In [27]:
file_dir = '/Users/cd201/mpl8-kin-mis/plots/pa_fits/'
preview = '/Applications/Preview.app/Contents/MacOS/Preview'
adobe = '/Applications/Adobe Acrobat Reader DC.app/Contents/MacOS/AdobeReader'

In [31]:
path = '/Users/cd201/mpl8-kin-mis/plots/pa_fits/10510-6103-PA.pdf'

In [16]:
img = subprocess.Popen([preview, path])

In [30]:
p = subprocess.Popen([adobe, path], stdout=subprocess.PIPE)

In [31]:
p.kill()

In [3]:
# Loading in csv containing all of the galaxies.
tab_raw = pd.read_csv('/Users/cd201/mpl8-kin-mis/catalogues/MPL8-PA-FIT-RAW.csv')

In [32]:
for pifu in tab_raw.plateifu[0:5]:
    path = file_dir + str(pifu) + '-PA.pdf'
    
    # Checking if file exists.
    if os.path.isfile(path) == True:
        p = subprocess.Popen([adobe, path], stdout=subprocess.PIPE)
        #p = subprocess.Popen(['open','-a','Preview',path], stdout=subprocess.PIPE, preexec_fn=os.setsid)
        #p = subprocess.Popen([preview, path], stdout=subprocess.PIPE, preexec_fn=os.setsid)
        con = input('Continue?')
        if con == 'y':
            p.kill()


Continue?y
Continue?y
Continue?y
Continue?y
Continue?y


In [25]:
p = subprocess.Popen(['open','-a','Preview',path], stdout=subprocess.PIPE, preexec_fn=os.setsid)
if input('Continue?') == 'y':
    os.killpg(os.getpgid(p.pid), signal.SIGTERM)

Continue?y


ProcessLookupError: [Errno 3] No such process

In [15]:
p = subprocess.call(['open','-a','Preview',path], stdout=subprocess.PIPE, preexec_fn=os.setsid)

In [39]:
tab_raw.plateifu[0:2]

0    10001-12701
1    10001-12702
Name: plateifu, dtype: object

In [33]:
# Preview seems to have some weird bug so using adobe. Trying a script which accepts input to save to dataframe.

In [56]:
import os 
import numpy as np
import pandas as pd
import subprocess
import time

In [57]:
# Loading in csv containing all of the galaxies.
tab_raw = pd.read_csv('/Users/cd201/mpl8-kin-mis/catalogues/MPL8-PA-FIT-RAW.csv')

In [59]:
file_dir = '/Users/cd201/mpl8-kin-mis/plots/pa_fits/'
adobe = '/Applications/Adobe Acrobat Reader DC.app/Contents/MacOS/AdobeReader'
outpath = '/Users/cd201/mpl8-kin-mis/catalogues/raw_classifications-'

In [ ]:
plateifus = tab_raw.plateifu
classifications = np.full(plateifus.shape, -99)

for i, pifu in enumerate(plateifus):
    # Finding file.
    path = file_dir + str(pifu) + '-PA.pdf'
    
    # Checking if file exists.
    if os.path.isfile(path) == True:
        # If it does, opening in adobe reader.
        p = subprocess.Popen([adobe, path], stdout=subprocess.PIPE)
        
        # Asking for classification.
        response = input('Please classify this galaxy. ID:'+str(pifu)+'. Type wut for more info on classifications. \n') 
        
        while response == 'wut':
            print('\n0: Dont use \n1: Messy \n2: Incoherent Gas rot but clean stellar \n3: Clean! \n4: KDC \n5: Warp')
            time.sleep(0.5)
            response = input('So whats it going to be? \n')
        
        try:
            response = int(response)          
        except ValueError:  
            while type(response) == str:
                try:
                    response = int(response)   
                except ValueError:
                    response = input('0-5 please. \n')
        
        classifications[i] = response
        p.kill()
        
    # If file doesn't exist, the classification is void. 
    else:
        classifications[i] = -99
        print(str(pifu)+' not found.')
    
    # Saving every 50 classifications.
    if i % 50 == 0:
        print(str(i)+' down. Backing up classifications.')
        tab_raw['classification'] = classifications
        tab_raw.to_csv(outpath+str(i)+'.csv', index=None)

In [61]:
classifications

array([ 3.,  3.,  3., ..., nan, nan, nan])